## 深度學習系列| 解讀LeNet及PyTorch實現
###  from [CSDN](https://blog.csdn.n

In [1]:
import torch
import torch.nn as nn


class LeModel(nn.Module):
    def __init__(self, num_class=10):
        super(LeModel, self).__init__()
        # CONV2d(in_ch, out_ch, k_size)
        self.conv1 = nn.Conv2d(1, 20, 5)   # 1x28x28 -> 20x24x24
        self.pool1 = nn.MaxPool2d(2)    # 20x24x24 -> 20x12x12
        self.conv2 = nn.Conv2d(20, 40, 5)    # 20x12x12 -> 40x8x8
        self.pool2 = nn.MaxPool2d(2)    # 16x8x8 -> 40x4x4
        self.FC1 = nn.Linear(640*1*1, 64)
        self.FC2 = nn.Linear(64, num_class)

    def forward(self, x):
        
    
        
        x = torch.relu(self.conv1(x))
        x = self.pool1(x)
        x = torch.relu(self.conv2(x))
        x = self.pool2(x)
        #x = torch.tanh(self.conv3(x))
        x = x.view(-1, 640*1*1)
        x = self.FC1(x)
        x = torch.relu(x)
        x = self.FC2(x)
        return x
    
model = LeModel(num_class=10)
print(model)

LeModel(
  (conv1): Conv2d(1, 20, kernel_size=(5, 5), stride=(1, 1))
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(20, 40, kernel_size=(5, 5), stride=(1, 1))
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (FC1): Linear(in_features=640, out_features=64, bias=True)
  (FC2): Linear(in_features=64, out_features=10, bias=True)
)


In [2]:
import torchsummary

# torchsummary.summary(model, input_size = (1, 28, 28))

In [3]:
import torch
import torch.optim as optim
#from LeNet import LeModel
import torchvision.transforms as transforms
import torch.utils.data
import torch.nn as nn


from torchvision.datasets import MNIST


#def main():
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#device = torch.device('cpu')
print(f'device is {device}')

transform = transforms.Compose([transforms.ToTensor(),
                               transforms.Normalize((0.1307,), (0.3081,))])

train_data = MNIST(root='./torch_v2', train=True,
                   transform=transform, download=True)
train_loader = torch.utils.data.DataLoader(train_data, batch_size=16, shuffle=False)

test_data = MNIST(root='./torch_v2', train=False,
                  transform=transform, download=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=16, shuffle=False)
#------------------

i = 0
for images, labels in train_loader:
    i = i + 1
print(i)
i = 0
for images, labels in test_loader:
    i = i + 1
print(i)

net = LeModel()
net.to(device)

optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.5)
loss_function = nn.CrossEntropyLoss()

## train ##
net.train()
# .train 是用來啟動 batch norm & drop out
for epoch in range(10):
    running_loss = 0.0
    for step, (images, labels) in enumerate(train_loader, start=0):
        optimizer.zero_grad()
        images, labels = images.to(device), labels.to(device)
        output = net(images)
        loss = loss_function(output, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if step % 500 == 499:
            print('epoch: %d | step: %5d | train_loss: %.5f |' % (epoch+1, step+1, running_loss/500))
            running_loss = 0.0

print('Finished Training !')

## evaluate ##
net.eval()
# .eval 是用來關閉 batch norm & drop out
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        output = net(images)
        _, predict = torch.max(output.data, 1)
        total += labels.size(0)
        correct += (predict == labels).sum().item()
print('Accuracy of the network on the 10000 test images: %d %%' % (
        100 * correct / total))
print("correct = ", correct)


# if __name__ == "__main__":
#     main()

device is cuda
3750
625
epoch: 1 | step:   500 | train_loss: 2.20033 |
epoch: 1 | step:  1000 | train_loss: 1.29783 |
epoch: 1 | step:  1500 | train_loss: 0.53149 |
epoch: 1 | step:  2000 | train_loss: 0.42091 |
epoch: 1 | step:  2500 | train_loss: 0.32488 |
epoch: 1 | step:  3000 | train_loss: 0.29735 |
epoch: 1 | step:  3500 | train_loss: 0.26630 |
epoch: 2 | step:   500 | train_loss: 0.21664 |
epoch: 2 | step:  1000 | train_loss: 0.22269 |
epoch: 2 | step:  1500 | train_loss: 0.18310 |
epoch: 2 | step:  2000 | train_loss: 0.18658 |
epoch: 2 | step:  2500 | train_loss: 0.15261 |
epoch: 2 | step:  3000 | train_loss: 0.16178 |
epoch: 2 | step:  3500 | train_loss: 0.15172 |
epoch: 3 | step:   500 | train_loss: 0.12899 |
epoch: 3 | step:  1000 | train_loss: 0.13624 |
epoch: 3 | step:  1500 | train_loss: 0.11543 |
epoch: 3 | step:  2000 | train_loss: 0.12134 |
epoch: 3 | step:  2500 | train_loss: 0.10465 |
epoch: 3 | step:  3000 | train_loss: 0.11568 |
epoch: 3 | step:  3500 | train_loss:

### Print test input 

In [4]:
index = 0
global img0
for images0, label0 in test_loader:
    if index > 0:
        break
    #print(images0[0])
    img0 = images0[0]
    index = index + 1
print("input size = ", img0.shape)

f = open("./para/input.txt", 'w')
for i in range(28):
    for j in range(28):
        print("%f," %float(img0[0][i][j]), end = '', file = f)
    print("",file = f)

f.close()

input size =  torch.Size([1, 28, 28])


### Show feature map 

In [5]:
## print layer intermediate output
# exact_list = ['conv1']
# myexactor = FeatureExtractor(net, exact_list)  # 输出是一个网络


def get_features(name):
    def hook(model, input, output):
        features[name] = output.detach()

    return hook

net.conv1.register_forward_hook(get_features("conv1"))
#net.FC2.register_forward_hook(get_features("FC2"))

# placeholders
PREDS = []
FEATS = []

# placeholder for batch features
features = {}
#net = net.to(device)
img0 = img0.to(device)

preds = net(img0)

PREDS.append(preds.detach().cpu().numpy())
FEATS.append(features["conv1"].cpu().numpy())

print(len(FEATS))
print(len(FEATS[0]))
print(len(FEATS[0][0]))
print(len(FEATS[0][0][0]))


    
f = open('./para/conv1_output.txt', 'w')
for out_ch in range(1):
    print("output channel =", out_ch)
    for row in range (24):
        for col in range(24):
            print("%f," % (float(FEATS[0][out_ch][row][col])),end='',file = f) 
            #print("%f," % (float(FEATS[0][out_ch][row][col])),end='') 
        print("\n\n",file = f)
f.close()
    
# net.FC2.register_forward_hook(get_features("FC2"))
# FEATS.append(features["FC2"].cpu().numpy())
# print(len(FEATS))
# print(len(FEATS[1]))
# print(FEATS[1])

# x = myexactor(img0)

# out = net.print_layer(img0)
# print("output size = ", out.shape)
# #print(output.type)
# #print(output)





1
20
24
24
output channel = 0


## print model weight

In [6]:
# for param in model.parameters():
#     print((param.data))
or_dict = net.state_dict()
#print(float(or_dict['conv1.weight'][0][0][0][0]))
#print(model.state_dict())
# for key, value in or_dict.iteritems() :
#     print(key)
print(or_dict.keys())

odict_keys(['conv1.weight', 'conv1.bias', 'conv2.weight', 'conv2.bias', 'FC1.weight', 'FC1.bias', 'FC2.weight', 'FC2.bias'])


In [7]:
print("kernel 1: ")
print((or_dict['conv1.weight'][0][0]))
print("bias 1: ")
print((or_dict['conv1.bias'][0]))

kernel 1: 
tensor([[ 0.1588, -0.1289,  0.2158,  0.2100,  0.2369],
        [-0.0027, -0.2277,  0.2486,  0.4137,  0.2132],
        [-0.0897, -0.1108,  0.0621,  0.4653,  0.4454],
        [-0.1639,  0.0008,  0.3310,  0.4311,  0.1824],
        [ 0.0968, -0.0749,  0.2344,  0.1288, -0.1309]], device='cuda:0')
bias 1: 
tensor(0.1770, device='cuda:0')


In [8]:
# for param in model.parameters():
#   print(param.data)

In [9]:
f = open('./para/conv1_weight.txt', 'w')
for i in range(5):
    for j in range(5):
        print("%f," % (or_dict['conv1.weight'][0][0][i][j]),end='',file = f) 
        print("%f," % (or_dict['conv1.weight'][0][0][i][j]),end='')
    print("\n", file = f)
f.close()




0.158808,-0.128901,0.215808,0.209981,0.236942,-0.002693,-0.227722,0.248634,0.413741,0.213184,-0.089750,-0.110789,0.062129,0.465253,0.445366,-0.163879,0.000763,0.331031,0.431078,0.182399,0.096818,-0.074894,0.234397,0.128761,-0.130873,

In [10]:
net(img0)

tensor([[ -0.0630,  -2.0820,   3.9497,   5.1293,  -5.6467,  -2.6576, -14.9247,
          13.5624,  -2.1464,   2.0591]], device='cuda:0',
       grad_fn=<AddmmBackward0>)

In [11]:
FILE = './MNIST/model/model_state_dict.pt'
torch.save(model.state_dict(), FILE)

In [12]:
torch.cuda.is_available()

True

In [13]:
torch.cuda.get_device_name()

'NVIDIA GeForce GTX 1650'

### print model weight
#### pytorch weight shape : out_ch, in_ch, height, width

In [14]:
f = open("./para/para.h", 'w')
conv_index = 1
fc_index = 1
print("#include \"../../inference/nn.h\"\n\n",file = f)
for layer_name in or_dict.keys():
    if 'conv' in layer_name and 'weight' in layer_name:
        print(layer_name)
        print("layer shape = ", or_dict[layer_name].shape)
        print("const double conv%d_w_raw[] = {" %conv_index, file = f)
        for out_ch in range (or_dict[layer_name].shape[0]):
            for in_ch in range (or_dict[layer_name].shape[1]):
                #print("%dth kernel, %dth channel" % (out_ch, in_ch), file = f)
                for h in range (or_dict[layer_name].shape[2]):
                    for w in range (or_dict[layer_name].shape[3]):
                        print("(%.15f), " % (float(or_dict[layer_name][out_ch][in_ch][h][w])),end='',file = f)
                    #print ("\n", file = f)
                #print("----------------\n", file = f)
        print("};\n\n", file = f)
    elif 'conv' in layer_name and 'bias' in layer_name:
        print(layer_name)
        print("layer shape = ", or_dict[layer_name].shape)
        print("const double conv%d_b_raw[] = {" %conv_index, file = f)
        conv_index = conv_index + 1
        for ch in range (or_dict[layer_name].shape[0]):
            print("(%.15f), " % (float(or_dict[layer_name][ch])),end='',file = f) 
        print("};\n\n", file = f)
    elif 'weight' in layer_name: # fc weight
        print(layer_name)
        print("layer shape = ", or_dict[layer_name].shape)
        print("const double fc%d_w_raw[] = {" %fc_index, file = f)
        for out_ch in range (or_dict[layer_name].shape[0]):
            for in_ch in range (or_dict[layer_name].shape[1]):
                 print("(%.15f), " % (float(or_dict[layer_name][out_ch][in_ch])),end='',file = f)
        print("};\n\n", file = f)
    elif 'bias' in layer_name: #fc bias
        print(layer_name)
        print("layer shape = ", or_dict[layer_name].shape)
        print("const double fc%d_b_raw[] = {" %fc_index, file = f)
        fc_index = fc_index + 1
        for out_ch in range (or_dict[layer_name].shape[0]):
            print("(%.15f), " % (float(or_dict[layer_name][out_ch])),end='',file = f)
        print("};\n\n", file = f)
        
f.close()
        

conv1.weight
layer shape =  torch.Size([20, 1, 5, 5])
conv1.bias
layer shape =  torch.Size([20])
conv2.weight
layer shape =  torch.Size([40, 20, 5, 5])
conv2.bias
layer shape =  torch.Size([40])
FC1.weight
layer shape =  torch.Size([64, 640])
FC1.bias
layer shape =  torch.Size([64])
FC2.weight
layer shape =  torch.Size([10, 64])
FC2.bias
layer shape =  torch.Size([10])
